In [19]:
import os
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, WeightedRandomSampler
import numpy as np

# Set root path
dataset_root = r"C:/Users/sangi/Downloads/IQ-Processed/IQ-Processed"

# Transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05), scale=(0.95, 1.05)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

])

# Load datasets
train_dataset = ImageFolder(root=os.path.join(dataset_root, "train"), transform=train_transform)
val_dataset = ImageFolder(root=os.path.join(dataset_root, "val"), transform=val_test_transform)
test_dataset = ImageFolder(root=os.path.join(dataset_root, "test"), transform=val_test_transform)

# Class balancing using WeightedRandomSampler
targets = train_dataset.targets
class_counts = np.bincount(targets)
class_weights = 1. / class_counts
sample_weights = [class_weights[label] for label in targets]

sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Print dataset info
print("Classes:", train_dataset.classes)
print("Train:", len(train_dataset), "| Val:", len(val_dataset), "| Test:", len(test_dataset))
print("Class counts in train:", dict(zip(train_dataset.classes, class_counts)))


Classes: ['Benign cases', 'Malignant cases', 'Normal cases']
Train: 823 | Val: 176 | Test: 178
Class counts in train: {'Benign cases': np.int64(140), 'Malignant cases': np.int64(392), 'Normal cases': np.int64(291)}


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import os
from tqdm import tqdm

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model getter
def get_model(name, num_classes=3):
    if name == 'mobilenet':
        model = models.mobilenet_v2(pretrained=True)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    elif name == 'efficientnet':
        model = models.efficientnet_b0(pretrained=True)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    elif name == 'googlenet':
        model = models.googlenet(pretrained=True, aux_logits=True)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif name == 'resnet':
        model = models.resnet18(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif name == 'densenet':
        model = models.densenet121(pretrained=True)
        model.classifier = nn.Linear(model.classifier.in_features, num_classes)
    else:
        raise ValueError("Unknown model name")
    return model

# Evaluation function
def evaluate(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            if isinstance(outputs, tuple):
                outputs = outputs[0]
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    acc = correct / total
    report = classification_report(all_labels, all_preds, target_names=['Benign', 'Malignant', 'Normal'], output_dict=True)
    conf_matrix = confusion_matrix(all_labels, all_preds)
    return report, conf_matrix, acc

# Training function with early stopping
def train_model(model, train_loader, val_loader, model_name, num_epochs=15, patience=5, lr=1e-4):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

    best_f1 = 0
    patience_counter = 0
    save_path = f"{model_name}_best.pth"

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - {model_name}")
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            if isinstance(outputs, tuple):
                outputs = outputs[0]
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            loop.set_postfix(loss=loss.item())

        train_acc = correct / total

        scheduler.step()

        # Validation
        report, _, val_acc = evaluate(model, val_loader)
        val_f1 = report['weighted avg']['f1-score']
        print(f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Val F1: {val_f1:.4f}")

        if val_f1 > best_f1:
            best_f1 = val_f1
            patience_counter = 0
            torch.save(model.state_dict(), save_path)
            print(f"✅ Saved Best Model: {save_path}")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("⏹️ Early stopping triggered.")
                break

    print(f"📈 Best Validation F1 for {model_name}: {best_f1:.4f}")

# After training: Test & Print Metrics
def test_model(model, test_loader, model_path):
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()
    report, conf_matrix, test_acc = evaluate(model, test_loader)

    print("\n=== Test Classification Report ===")
    for label, metrics in report.items():
        if isinstance(metrics, dict):
            print(f"{label}: F1={metrics['f1-score']:.4f}, Precision={metrics['precision']:.4f}, Recall={metrics['recall']:.4f}")
    print(f"\nTest Accuracy: {test_acc:.4f}")
    print("\nConfusion Matrix:\n", conf_matrix)

# ========= Main Script to Train All Models ==========

model_names = ['mobilenet', 'efficientnet', 'googlenet', 'resnet', 'densenet']

# You'll already have these from your earlier data loading code
# train_loader, val_loader, test_loader

for name in model_names:
    print(f"\n==== Training {name.upper()} ====")
    model = get_model(name)
    train_model(model, train_loader, val_loader, name)

    print(f"\n==== Evaluating {name.upper()} on Test Data ====")
    test_model(model, test_loader, f"{name}_best.pth")



==== Training MOBILENET ====


C:\Users\sangi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\sangi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\sangi/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100.0%
Epoch 1/15 - m

Train Acc: 0.6865 | Val Acc: 0.7898 | Val F1: 0.7778
✅ Saved Best Model: mobilenet_best.pth


Epoch 2/15 - mobilenet: 100%|██████████| 26/26 [01:41<00:00,  3.91s/it, loss=0.286]


Train Acc: 0.7764 | Val Acc: 0.8580 | Val F1: 0.8526
✅ Saved Best Model: mobilenet_best.pth


...

Train Acc: 0.9830 | Val Acc: 0.9659 | Val F1: 0.9652


Epoch 14/15 - densenet: 100%|██████████| 26/26 [04:26<00:00, 10.25s/it, loss=0.226] 


Train Acc: 0.9854 | Val Acc: 0.9716 | Val F1: 0.9719


Epoch 15/15 - densenet: 100%|██████████| 26/26 [04:11<00:00,  9.69s/it, loss=0.0486]


Train Acc: 0.9842 | Val Acc: 0.9886 | Val F1: 0.9887
✅ Saved Best Model: densenet_best.pth
📈 Best Validation F1 for densenet: 0.9887

==== Evaluating DENSENET on Test Data ====

=== Test Classification Report ===
Benign: F1=0.9231, Precision=0.8571, Recall=1.0000
Malignant: F1=1.0000, Precision=1.0000, Recall=1.0000
Normal: F1=0.9587, Precision=1.0000, Recall=0.9206
macro avg: F1=0.9606, Precision=0.9524, Recall=0.9735
weighted avg: F1=0.9724, Precision=0.9759, Recall=0.9719

Test Accuracy: 0.9719

Confusion Matrix:
 [[30  0  0]
 [ 0 85  0]
 [ 5  0 58]]


In [20]:
# Load all trained CNN models
model_names = ['mobilenet', 'efficientnet', 'googlenet', 'resnet', 'densenet']
trained_models = {}

for name in model_names:
    print(f"Loading {name.upper()}...")
    model = get_model(name)
    model.load_state_dict(torch.load(f"{name}_best.pth"))
    trained_models[name] = model


Loading MOBILENET...
Loading EFFICIENTNET...
Loading GOOGLENET...


C:\Users\sangi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\googlenet.py:341: UserWarning: auxiliary heads in the pretrained googlenet model are NOT pretrained, so make sure to train them
  warnings.warn(


Loading RESNET...
Loading DENSENET...


In [21]:
import torch
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import os

# Set your device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create directory for saved models if it doesn't exist
os.makedirs("saved_models", exist_ok=True)

# ML classifiers
ml_models = {
    "svm": SVC(kernel="linear", probability=True),
    "knn": KNeighborsClassifier(n_neighbors=5),
    "naive_bayes": GaussianNB(),
    "decision_tree": DecisionTreeClassifier(),
    "random_forest": RandomForestClassifier(n_estimators=100),
    "adaboost": AdaBoostClassifier(n_estimators=50),
    "mlp": MLPClassifier(hidden_layer_sizes=(100,), max_iter=500)
}

# Custom CNN weights
cnn_weights = {
    "mobilenet": 1.0,
    "resnet": 0.7,
    "efficientnet": 1.5,
    "googlenet": 1.0,
    "densenet": 0.7,
}

# Feature extraction function
def extract_features(model, dataloader):
    model.eval()
    features, labels = [], []

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)

            if isinstance(outputs, tuple):  # for models like GoogLeNet
                outputs = outputs[0]

            outputs = outputs.view(outputs.size(0), -1)  # Flatten features
            features.append(outputs.cpu().numpy())
            labels.append(targets.numpy())

    features = np.vstack(features)
    labels = np.hstack(labels)
    return features, labels

# Dictionary to store prediction probabilities
final_probabilities = {}

# Train ML models on CNN features and save them
for cnn_name, cnn_model in trained_models.items():
    print(f"\n--- Extracting features from {cnn_name.upper()} ---")
    cnn_model.to(device)
    X_features, y_labels = extract_features(cnn_model, val_loader)

    print(f"--- Training and saving ML models on {cnn_name.upper()} features ---")
    model_probabilities = []

    for ml_name, ml_model in ml_models.items():
        # Train the model
        ml_model.fit(X_features, y_labels)
        
        # Save the trained ML model
        ml_model_path = f"saved_models/{cnn_name}_{ml_name}_model.pkl"
        joblib.dump(ml_model, ml_model_path)
        print(f"Saved {ml_name} model for {cnn_name} at {ml_model_path}")
        
        # Get predictions
        y_prob = ml_model.predict_proba(X_features)
        acc = accuracy_score(y_labels, np.argmax(y_prob, axis=1))
        print(f"{ml_name} on {cnn_name}: {acc:.4f}")
        model_probabilities.append(y_prob)

    final_probabilities[cnn_name] = np.array(model_probabilities)

    # Save the features for later use if needed
    np.savez(f"saved_models/{cnn_name}_features.npz", 
             features=X_features, 
             labels=y_labels)

# Save the CNN models if they're not already saved
for cnn_name, cnn_model in trained_models.items():
    model_path = f"saved_models/{cnn_name}_model.pth"
    if not os.path.exists(model_path):
        torch.save(cnn_model.state_dict(), model_path)
        print(f"Saved {cnn_name} model at {model_path}")

# Ensemble Voting
voted_probabilities = np.zeros_like(final_probabilities["googlenet"][0])

for cnn_name, probs in final_probabilities.items():
    weight = cnn_weights[cnn_name]
    avg_prob = np.mean(probs, axis=0) * weight
    voted_probabilities += avg_prob

# Normalize and predict final class
voted_probabilities /= sum(cnn_weights.values())
voted_predictions = np.argmax(voted_probabilities, axis=1)

# Final ensemble accuracy
ensemble_accuracy = accuracy_score(y_labels, voted_predictions)
print(f"\n✅ Final Weighted Ensemble Accuracy (Validation Set): {ensemble_accuracy:.4f}")

# Save the ensemble voting weights for deployment
ensemble_weights = {
    'cnn_weights': cnn_weights,
    'class_names': ['Benign cases', 'Malignant cases', 'Normal cases']
}
joblib.dump(ensemble_weights, 'saved_models/ensemble_weights.pkl')
print("Saved ensemble weights configuration")


--- Extracting features from MOBILENET ---
--- Training and saving ML models on MOBILENET features ---
Saved svm model for mobilenet at saved_models/mobilenet_svm_model.pkl
svm on mobilenet: 0.9545
Saved knn model for mobilenet at saved_models/mobilenet_knn_model.pkl
knn on mobilenet: 0.9602
Saved naive_bayes model for mobilenet at saved_models/mobilenet_naive_bayes_model.pkl
naive_bayes on mobilenet: 0.9375
Saved decision_tree model for mobilenet at saved_models/mobilenet_decision_tree_model.pkl
decision_tree on mobilenet: 1.0000
Saved random_forest model for mobilenet at saved_models/mobilenet_random_forest_model.pkl
random_forest on mobilenet: 1.0000
Saved adaboost model for mobilenet at saved_models/mobilenet_adaboost_model.pkl
adaboost on mobilenet: 1.0000
Saved mlp model for mobilenet at saved_models/mobilenet_mlp_model.pkl
mlp on mobilenet: 0.9602

--- Extracting features from EFFICIENTNET ---
--- Training and saving ML models on EFFICIENTNET features ---
Saved svm model for ef

In [33]:
import streamlit as st
import torch
import torch.nn as nn
from torchvision import models, transforms
import numpy as np
import joblib
from PIL import Image
import os
from sklearn.metrics import classification_report

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Class names
class_names = ['Benign cases', 'Malignant cases', 'Normal cases']

# CNN Weights
cnn_weights = {
    "mobilenet": 1.0,
    "resnet": 0.7,
    "efficientnet": 1.5,
    "googlenet": 1.0,
    "densenet": 0.7,
}

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load all CNN models
def get_model(name, num_classes=3):
    if name == 'mobilenet':
        model = models.mobilenet_v2(pretrained=False)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    elif name == 'efficientnet':
        model = models.efficientnet_b0(pretrained=False)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    elif name == 'googlenet':
        model = models.googlenet(pretrained=False, aux_logits=True)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif name == 'resnet':
        model = models.resnet18(pretrained=False)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif name == 'densenet':
        model = models.densenet121(pretrained=False)
        model.classifier = nn.Linear(model.classifier.in_features, num_classes)
    else:
        raise ValueError("Unknown model name")
    return model

@st.cache_resource
def load_cnn_models():
    model_dict = {}
    for name in cnn_weights.keys():
        model = get_model(name)
        model_path = f"saved_models/{name}_model.pth"
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.to(device)
        model.eval()
        model_dict[name] = model
    return model_dict

def extract_features_single(model, image_tensor):
    with torch.no_grad():
        output = model(image_tensor)
        if isinstance(output, tuple):  # GoogLeNet
            output = output[0]
        output = output.view(output.size(0), -1)
        return output.cpu().numpy()

def ensemble_predict(image):
    image_tensor = transform(image).unsqueeze(0).to(device)
    cnn_models = load_cnn_models()
    total_prob = np.zeros((1, 3))

    for cnn_name, model in cnn_models.items():
        features = extract_features_single(model, image_tensor)
        probs_list = []

        for ml_name in ['svm', 'knn', 'naive_bayes', 'decision_tree', 'random_forest', 'adaboost', 'mlp']:
            ml_model_path = f"saved_models/{cnn_name}_{ml_name}_model.pkl"
            if os.path.exists(ml_model_path):
                ml_model = joblib.load(ml_model_path)
                probs = ml_model.predict_proba(features)
                probs_list.append(probs)

        if probs_list:
            avg_prob = np.mean(probs_list, axis=0)
            weighted_prob = avg_prob * cnn_weights[cnn_name]
            total_prob += weighted_prob

    # Normalize
    total_prob /= sum(cnn_weights.values())
    final_pred = np.argmax(total_prob)
    return final_pred, total_prob[0]

# ===== Streamlit UI =====
st.set_page_config(page_title="Lung Cancer Detection - Ensemble App", layout="centered")
st.title("🫁 Lung Cancer Detection using Ensemble Learning")
st.write("Upload a CT scan image to detect possible lung cancer.")

uploaded_image = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])

if uploaded_image is not None:
    image = Image.open(uploaded_image).convert("RGB")
    st.image(image, caption="Uploaded Image", use_column_width=True)

    if st.button("🔍 Predict"):
        with st.spinner("Analyzing with ensemble model..."):
            pred_class, pred_probs = ensemble_predict(image)
            st.success(f"🎯 **Prediction**: {class_names[pred_class]}")
            st.write("### 🧪 Prediction Probabilities:")
            for i, cls in enumerate(class_names):
                st.write(f"{cls}: {pred_probs[i]*100:.2f}%")

            # Optional: Display mock classification report structure
            predicted_label = pred_class
            true_label = pred_class  # Assume prediction for uploaded unknown
            report = classification_report(
    [true_label], [predicted_label],
    labels=[0, 1, 2],
    target_names=class_names,
    output_dict=True,
    zero_division=0
)

            st.write("### 📊 Classification Report")
            for label in class_names:
                st.write(f"**{label}** — Precision: {report[label]['precision']:.2f}, Recall: {report[label]['recall']:.2f}, F1: {report[label]['f1-score']:.2f}")


2025-04-15 18:51:06.514 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 18:51:06.516 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 18:51:06.517 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 18:51:06.518 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 18:51:06.519 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 18:51:06.520 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 18:51:06.520 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 18:51:06.521 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [34]:
!streamlit run sdp_app.py & npx localtunnel --port 8501

^C
